In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [4]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.svm import SVC
from ml_lib import KernelizedSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [19]:
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)

print("Shape of X:", X)
np.sum(X**2, axis=1)

Shape of X: [[ 1.58202308 -0.44581483]
 [ 0.0660451   0.4392075 ]
 [ 0.73663111 -0.39896339]
 [-1.05692838  0.2424558 ]
 [-0.80216162  0.20271838]
 [-0.70641308  0.77076218]
 [ 0.2053884   0.81875305]
 [ 1.37804958 -0.44658032]
 [ 0.8481931   0.67172277]
 [ 0.92202981 -0.08438964]
 [ 0.38972996  0.81785686]
 [-0.33031615  0.9271263 ]
 [ 1.1673298  -0.5978882 ]
 [-0.59659735  0.89753707]
 [ 1.92535861  0.1831877 ]
 [ 1.12958691  0.33602789]
 [ 1.12532895 -0.40571607]
 [ 1.09856769 -0.51879837]
 [ 0.57010693  0.6727978 ]
 [-0.30851921  0.89344338]
 [-0.02646004  0.14891397]
 [ 1.83385428 -0.17380486]
 [-0.01103091  0.86424669]
 [-0.98799936  0.38321942]
 [ 0.52363988 -0.14410208]
 [ 0.46249008  0.91166069]
 [-0.90469417  0.49415615]
 [ 2.01169144  0.16298968]
 [ 0.77181427  0.53088762]
 [ 0.7557984  -0.56943072]
 [ 0.87011866  0.74872708]
 [-1.03596662  0.11946065]
 [ 1.74719141 -0.42447937]
 [-0.18871867  1.17238503]
 [ 2.12342023 -0.06925813]
 [ 0.87279305  0.33197416]
 [ 0.34941974 -0

array([2.70154789e+00, 1.97265182e-01, 7.01797184e-01, 1.17588242e+00,
       6.84558006e-01, 1.09309378e+00, 7.12540946e-01, 2.09845462e+00,
       1.17064300e+00, 8.57260585e-01, 8.20779276e-01, 9.68671927e-01,
       1.72012915e+00, 1.16150119e+00, 3.74056353e+00, 1.38888134e+00,
       1.43097078e+00, 1.47600273e+00, 7.77678785e-01, 8.93425178e-01,
       2.28755034e-02, 3.39322966e+00, 7.47044023e-01, 1.12299986e+00,
       2.94964130e-01, 1.04502228e+00, 1.06266184e+00, 4.07346807e+00,
       8.77538939e-01, 8.95482572e-01, 1.31769873e+00, 1.08749768e+00,
       3.23286056e+00, 1.41010140e+00, 4.51371018e+00, 8.71974550e-01,
       3.00150667e-01, 2.03938171e-01, 1.04007444e+00, 1.15165900e+00,
       1.00871818e+00, 9.88519807e-01, 7.59848164e-01, 4.32150902e+00,
       9.75614091e-01, 1.10991109e+00, 2.50625274e+00, 1.18837596e+00,
       2.06955310e-01, 9.87614914e-01, 1.04221911e+00, 1.21247651e+00,
       8.05028954e-01, 9.45749704e-01, 1.06696463e+00, 1.79831316e+00,
      